In [1]:
import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt 
import os
import math

In [3]:
#I made lists of all of the data for each well respectively and plotted some of them to look at how the data correlates
lengthsOfWells = []
oilSaturationAverage = []
thickness = []
waterSaturation = []
permeability = []
porosity = []
Young = []
names = []
poisson = []
northing = []
easting = []

# making a list of the means of each category for each well 
for elem in os.listdir("data"):
    if elem != "well production.csv":
        df = pd.read_csv("data/" + elem)
        oilSaturationAverage.append(df["oil saturation"].values.mean())
        thickness.append(df["thickness (ft)"].values.mean())
        waterSaturation.append(df["water saturation"].values.mean())
        permeability.append(df["permeability"].values.mean())
        porosity.append(df["porosity"].values.mean())
        Young.append(df["Young's Modulus"].values.mean())
        lengthsOfWells.append(df.iloc[-1][0] - df.iloc[0][0])
        poisson.append(df["Poisson's ratio"].values.mean())
        northing.append(df["northing"].values.mean())
        easting.append(df["easting"].values.mean())
        names.append(elem)
        
        
# making a compiled dataframe that contains the data for all 100 wells        
dfCompiled = pd.DataFrame(list(zip(names, northing, easting, lengthsOfWells, thickness, porosity, permeability, waterSaturation, Young, oilSaturationAverage, poisson)),
                          columns =["Name", "northing", "CENTER of easting", 'Length', 'Thickness', "Porosity", "Permeability", "water saturation", "Young's modulus", "oil saturation", "poisson's ratio"])


# getting data (pressure and recovery factor) from the well production.csv
dfWellProduction = pd.read_csv("data/well production.csv")
dfWellProduction = dfWellProduction.sort_values(by=["well name"])
dfWellProduction = dfWellProduction.reset_index(drop=True)


dfCompiled['average pressure (Pa)'] = dfWellProduction['average pressure (Pa)']
dfCompiled['recovery factor'] = dfWellProduction["recovery factor"]


                         
              

In [4]:
# now doing the calculations

# calculating E' (where E is Young's modulus and v is poisson's ratio)
def E_prime(E, v):
    return E / (1 - pow(v,2))

# calculating b = half minor axis of an ellipse (we are using the max fracture width divided by 2 to estimate b)
# where H is the thickness in feet and p is the average pressure
def b(H, E_prime, p):
    return (H * p) / E_prime

# where L is the length of the well and b is the half minor axis of an ellipse
def areaMethod1(L, b):
  return (L * 2 * b + math.pi * b * b) / 43650  

#second area function
def areaMethod2(L, b):
    return (math.pi * (L / 2 + b) * b) / 43560
# the original oil in place formula where A is area (in acres), H is thickness, por is porosity, s is water saturation, 
# and fvf is formation volume factor
def oil_in_place(A, H, por, s):
    return (7758 * A * H * por * (1 - s)) / 1.6

# calculate recoverable reserves where OOIP is the original oil in place and rf is the recovery factor
def r_r(OOIP, rf):
    return OOIP * rf
    

dfCompiled["E prime calculation"] = E_prime(dfCompiled["Young's modulus"], dfCompiled["poisson's ratio"])
dfCompiled["b calculation"] = b(dfCompiled["Thickness"], dfCompiled["E prime calculation"], dfCompiled["average pressure (Pa)"])
dfCompiled["Area method one"] = areaMethod1(dfCompiled["Length"], dfCompiled["b calculation"])
dfCompiled["Area method two"] = areaMethod2(dfCompiled["Length"], dfCompiled["b calculation"])
dfCompiled["Area average"] = (dfCompiled["Area method one"] + dfCompiled["Area method two"])/2
dfCompiled["original oil in place"] = oil_in_place(dfCompiled["Area average"], dfCompiled["Thickness"], 
                                                   dfCompiled["Porosity"], dfCompiled["water saturation"])
dfCompiled["recoverable reserves"] = r_r(dfCompiled["original oil in place"], dfCompiled["recovery factor"])

display(dfCompiled)                                           


,Name,northing,CENTER of easting,Length,Thickness,Porosity,Permeability,water saturation,Young's modulus,oil saturation,poisson's ratio,average pressure (Pa),recovery factor,E prime calculation,b calculation,Area method one,Area method two,Area average,original oil in place,recoverable reserves
0,Federal 14-113H.csv,31000.0,84831.86,10664.0,57.315355,0.0717,0.02261,0.071061,1.027616e+07,0.928939,0.30560,35023133,0.107214,1.133472e+07,177.098567,88.790168,70.365146,79.577657,1.472985e+06,157925.151436
1,Gunnison 8-30H.csv,56300.0,21846.69,4893.0,60.079685,0.0260,0.00303,0.037063,1.031733e+07,0.962937,0.29640,36087449,0.072103,1.131103e+07,191.682085,45.618084,36.471052,41.044568,2.993535e+05,21584.434331
2,Gunnison 4-93H.csv,45200.0,31664.47,6929.0,95.127367,0.0418,0.00724,0.077795,1.237790e+07,0.922205,0.29326,36199370,0.097415,1.354259e+07,254.275706,85.380893,68.197249,76.789071,1.365333e+06,133003.455633
3,King 14-181H.csv,91600.0,88244.50,8489.0,66.136285,0.0299,0.00418,0.133591,7.493251e+06,0.866409,0.28326,31397763,0.085264,8.146930e+06,254.885135,103.815301,82.710316,93.262808,7.747685e+05,66060.236743
4,State 11-28H.csv,31500.0,43087.62,6375.0,111.757014,0.0673,0.01961,0.213838,1.080265e+07,0.786162,0.33057,30569755,0.079498,1.212795e+07,281.695061,87.993219,70.480659,79.236939,2.271744e+06,180598.099751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Burger 2-179H.csv,73300.0,53866.67,7533.0,49.302468,0.0444,0.00815,0.000332,1.003238e+07,0.999668,0.31283,36454201,0.102102,1.112067e+07,161.616296,57.662519,45.785872,51.724196,5.488211e+05,56035.953140
96,Golden 14-27H.csv,63400.0,20653.30,8707.0,20.638622,0.0212,0.00197,0.047221,9.179936e+06,0.952779,0.29973,36269180,0.107993,1.008605e+07,74.215981,30.004605,23.699500,26.852053,5.427701e+04,5861.557956
97,Golden 11-109H.csv,42400.0,90124.95,6450.0,23.921412,0.0578,0.01357,0.124730,1.094887e+07,0.875270,0.29506,34518701,0.102955,1.199299e+07,68.851567,20.689073,16.356104,18.522589,1.086896e+05,11190.100945
98,Burger 3-45H.csv,64800.0,61374.14,8948.0,54.655185,0.0579,0.01432,0.046641,9.333535e+06,0.953359,0.32993,35955602,0.085654,1.047363e+07,187.629331,79.459646,63.081276,71.270461,1.042572e+06,89300.497859
